In [37]:
# Library
import pandas as pd
from pymongo import MongoClient

# Conectando ao MongoDB
connection_string = "mongodb://localhost:27017"
client = MongoClient(connection_string)
db_connection = client["avaliação_dataops"]

collection_car = db_connection.get_collection("Carros")
collection_mon = db_connection.get_collection("Montadoras")

# Dados para a tabela "Carros"
carros_data = [
    {'Carro': "Onix", 'Cor': 'Prata', 'Montadora': "Chevrolet"},
    {'Carro': "Polo", 'Cor': 'Branco', 'Montadora': "Volkswagen"},
    {'Carro': "Sandero", 'Cor': 'Prata', 'Montadora': "Renault"},
    {'Carro': "Fiesta", 'Cor': 'Vermelho', 'Montadora': "Ford"},
    {'Carro': "City", 'Cor': 'Preto', 'Montadora': "Honda"}
]

# Inserindo os documentos na coleção "Carros"
collection_car.insert_many(carros_data)

# Dados para a tabela "Montadora"
montadora_data = [
    {'Montadora': "Chevrolet", 'País': "EUA"},
    {'Montadora': "Volkswagen", 'País': "Alemanhã"},
    {'Montadora': "Renault", 'País': "França"},
    {'Montadora': "Ford", 'País': "EUA"},
    {'Montadora': "Honda", 'País': "Japão"}
]

# Inserindo os documentos na coleção "Montadora"
collection_mon.insert_many(montadora_data)

In [45]:
# Consulta para recuperar todos os documentos (registros) da coleção "Carros"
cursor_car = collection_car.find({})

# Transformar os documentos em uma lista de dicionários
carros_data = list(cursor_car)

# Criar o DataFrame a partir dos dados
df_carros = pd.DataFrame(carros_data)

print(df_carros)

                        _id    Carro       Cor   Montadora
0  64ba9f00ce2a4b469de730ed     Onix     Prata   Chevrolet
1  64ba9f00ce2a4b469de730ee     Polo    Branco  Volkswagen
2  64ba9f00ce2a4b469de730ef  Sandero     Prata     Renault
3  64ba9f00ce2a4b469de730f0   Fiesta  Vermelho        Ford
4  64ba9f00ce2a4b469de730f1     City     Preto       Honda


In [46]:
# Consulta para recuperar todos os documentos (registros) da coleção "Montadora"
cursor_mon = collection_mon.find({})

# Transformar os documentos em uma lista de dicionários
montadora_data = list(cursor_mon)

# Criar o DataFrame a partir dos dados
df_montadora = pd.DataFrame(montadora_data)

print(df_montadora)

                        _id   Montadora      País
0  64baaab3ce2a4b469de730f2   Chevrolet       EUA
1  64baaab3ce2a4b469de730f3  Volkswagen  Alemanhã
2  64baaab3ce2a4b469de730f4     Renault    França
3  64baaab3ce2a4b469de730f5        Ford       EUA
4  64baaab3ce2a4b469de730f6       Honda     Japão


In [52]:
#Merge para adicionar a coluna "País"
df_merged = pd.merge(df_carros, df_montadora, on="Montadora")

df_merged

,_id_x,Carro,Cor,Montadora,_id_y,País
0,64ba9f00ce2a4b469de730ed,Onix,Prata,Chevrolet,64baaab3ce2a4b469de730f2,EUA
1,64ba9f00ce2a4b469de730ee,Polo,Branco,Volkswagen,64baaab3ce2a4b469de730f3,Alemanhã
2,64ba9f00ce2a4b469de730ef,Sandero,Prata,Renault,64baaab3ce2a4b469de730f4,França
3,64ba9f00ce2a4b469de730f0,Fiesta,Vermelho,Ford,64baaab3ce2a4b469de730f5,EUA
4,64ba9f00ce2a4b469de730f1,City,Preto,Honda,64baaab3ce2a4b469de730f6,Japão


In [68]:
aggregated_data = df_merged.groupby('País')['Carro'].count().reset_index()

# Renomear a coluna resultante para "Quantidade de Carros"
aggregated_data.rename(columns={'Carro': 'Quantidade de Carros'}, inplace=True)

print(aggregated_data)

       País  Quantidade de Carros
0  Alemanhã                     1
1       EUA                     2
2    França                     1
3     Japão                     1


In [70]:
# Converter o DataFrame para formato JSON
json_data = aggregated_data.to_json(orient='records')

# Caminho e nome do arquivo .js que você deseja salvar
file_path = "aggregated_data.js"

# Escrever o JSON no arquivo .js
with open(file_path, "w") as file:
    file.write(f"var aggregatedData = {json_data};")